In [ ]:
from pyAudioAnalysis import ShortTermFeatures as aF
from pyAudioAnalysis import audioBasicIO 
import numpy as np 
import plotly.graph_objs as go 
import plotly
import IPython
from pyAudioAnalysis import audioSegmentation as aS
# Load the API
from inaSpeechSegmenter import Segmenter
from inaSpeechSegmenter.export_funcs import seg2csv, seg2textgrid
from pydub import AudioSegment

In [ ]:
# read audio data from file 
# (returns sampling freq and signal as a numpy array)
fs, s = audioBasicIO.read_audio_file("18copy.wav")

In [ ]:
IPython.display.display(IPython.display.Audio("18copy.wav"))

In [ ]:
# print duration in seconds:
duration = len(s) / float(fs)
print(f'duration = {duration} seconds')

In [ ]:
# extract short-term features using a 50msec non-overlapping windows
win, step = 0.070, 0.070
[f, fn] = aF.feature_extraction(s, fs, int(fs * win), 
                                int(fs * step))

In [ ]:
input_media = '18copy.wav'
output_media = '18copy_cleaned.wav'
output_path = 'audios_extracted'

In [ ]:
seg = Segmenter()
segmentation = seg(input_media)

In [ ]:
print(segmentation)

In [ ]:
seg2csv(segmentation, 'myseg.csv')
# Here is the resulting CSV
# !cat myseg.csv

In [ ]:
import pandas as pd
df = pd.read_csv('myseg.csv', sep='\t')
df = df[(df['labels'] != 'noEnergy' ) & (df['labels'] != 'noise')]
df = df.reset_index(drop=True)
display(df)

In [ ]:
from pydub import AudioSegment
def cut_audio_from_wav(input_path, file_name, start, end, output_path, output_file):
  startSec = start*1000
  endSec = end*1000
  # Opening file and extracting segment
  song = AudioSegment.from_wav(input_path + file_name)
  extract = song[startSec:endSec]
  # Saving
  extract.export(f"{output_path}/{output_file}", format="wav")

with open('extract_audios.txt', 'w', encoding='utf-8') as  arq:
  total_lines = df.shape[0] -1
  for index, row in df.iterrows():
    try:
      
      output_file = f"_18copy_{index}.wav"
      cut_audio_from_wav('', '18copy.wav', row['start'], row['stop'],output_path, output_file)
      if index == total_lines:
        arq.write(f"file '{output_path}/{output_file}'")
      else:
        arq.write(f"file '{output_path}/{output_file}'\n")
    except Exception as e:
      print(e)

In [ ]:
import subprocess
txt_full_directory='extract_audios.txt'

# cmdargs = "ffmpeg -f concat -safe 0 -i " + txt_full_directory + " -c copy " + f"{output_media}"
cmd_args = f"ffmpeg -f concat -safe 0 -i {txt_full_directory} -c copy {output_media}"
# subprocess.run() will allow the process to end before creating a new one
try:
  subprocess.run(cmd_args, check=True, shell=True)
  print(f"File {output_media} successfully created")
except Exception as e:
  print(e)

In [ ]:
import speech_recognition as sr
import sys
from pydub import AudioSegment

In [ ]:
r = sr.Recognizer()
with sr.AudioFile(output_media) as source:
   audio_text = r.record(source)
   print(audio_text)
   text = r.recognize_google(audio_text,language='pt-BR')

In [ ]:
text

In [ ]:

with open(f'{output_media}_trancribed.txt', 'w') as arq:
  arq.write(text)